In [1]:
import os
import cv2
import torch
import json
import numpy as np
import open3d as o3d
import rosbag
import matplotlib.pyplot as plt
import custom_utils.data_conversion as data_conversion
import custom_utils.depth_anything_interface as depth_anything_interface

MODEL_TYPE = "metric"
DATASET = "hypersim"
ENCODER = "vitl"
DATAFILE = "/scratchdata/moving_2L"
CAMERA_JSON = "/scratchdata/gemini_2l.json"
MODEL_PATH = f"/scratchdata/depth_anything_v2_metric_{DATASET}_{ENCODER}.pth"
with open(CAMERA_JSON, 'r') as f:
    CAMERA_DATA = json.load(f)

DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
MODEL = depth_anything_interface.get_model(DEVICE, MODEL_PATH, model_type = MODEL_TYPE, encoder=ENCODER)

# Open bag file
bag_file_path = os.path.join(DATAFILE, "raw.bag")
bag = rosbag.Bag(bag_file_path)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2024-08-26 05:30:47,962 - topics - topicmanager initialized
WARNING - 2024-08-26 05:30:49,442 - attention - xFormers not available
WARNING - 2024-08-26 05:30:49,444 - block - xFormers not available
INFO - 2024-08-26 05:30:49,473 - dinov2 - using MLP layer as FFN
/depth-stereo/scripts/custom_utils/depth_anything_interface.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`

In [2]:
if not os.path.exists(os.path.join(DATAFILE, "depth")):
    os.makedirs(os.path.join(DATAFILE, "depth"))
if not os.path.exists(os.path.join(DATAFILE, "est_depth")):
    os.makedirs(os.path.join(DATAFILE, "est_depth"))

#Remove directory
if os.path.exists(os.path.join(DATAFILE, "depth")):
    for file in os.listdir(os.path.join(DATAFILE, "depth")):
        os.remove(os.path.join(DATAFILE, "depth", file))
if os.path.exists(os.path.join(DATAFILE, "est_depth")):
    for file in os.listdir(os.path.join(DATAFILE, "est_depth")):
        os.remove(os.path.join(DATAFILE, "est_depth", file))



In [3]:
for topic, msg, t in bag.read_messages(topics=["/camera/color/camera_info"]):
    D = np.array(msg.D)
    K = np.array(msg.K).reshape((3, 3))
    P = np.array(msg.P).reshape((3, 4))
    R = np.array(msg.R).reshape((3, 3))
    height = msg.height
    width = msg.width
    distortion_model = msg.distortion_model
    break

In [4]:
depth = None
img = None
cnt = 0

for topic, msg, t in bag.read_messages(topics=["/camera/color/image_raw", "/camera/depth/image_raw"]):
    if topic == "/camera/color/image_raw":
        img = data_conversion.topic_to_image(msg)
    elif topic == "/camera/depth/image_raw":
        depth = data_conversion.topic_to_depth(msg,CAMERA_DATA)
        
    if depth is not None and img is not None:
        
        est_depth = MODEL.infer_image(np.array(img)) # HxW raw depth map in numpy
        
        HEIGHT, WIDTH, _ = img.shape
        print(WIDTH, HEIGHT)

        pcd_depth = o3d.geometry.PointCloud()
        pcd_depth_est = o3d.geometry.PointCloud()

        for i in range(HEIGHT):
            for j in range(WIDTH):
                if depth[i, j] == 0:
                    continue
                x = (j - K[0, 2]) * depth[i, j] / K[0, 0]
                y = (i - K[1, 2]) * depth[i, j] / K[1, 1]
                z = depth[i, j]
                
                pcd_depth.points.append([x, y, z])
                pcd_depth.colors.append(img[i, j] / 255.0)
                
                if est_depth[i, j] == 0:
                    continue
                x = (j - K[0, 2]) * est_depth[i, j] / K[0, 0]
                y = (i - K[1, 2]) * est_depth[i, j] / K[1, 1]
                z = est_depth[i, j]
                
                pcd_depth_est.points.append([x, y, z])
                pcd_depth_est.colors.append(img[i, j] / 255.0)
        
        o3d.io.write_point_cloud(os.path.join(DATAFILE, "depth", f"{cnt:06d}.ply"), pcd_depth)
        o3d.io.write_point_cloud(os.path.join(DATAFILE, "est_depth", f"{cnt:06d}.ply"), pcd_depth_est)

        depth = None
        img = None
        cnt += 1


640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
640 400
